This blog will give a quick demonstration on a ***text mining example on the Consumer Financial complaints data*** published on the Consumer Financial Protection Bureau (CFPB) website.

This blog utilizes ***the text attribute - Consumer complaint narrative*** to answer one simple question:
what are the top 10 key complaint word that the top 3 credit bureau agencies (Equifax, Experian, and TransUnion) received ?

By extracting those keywords, it might help financial agencies like credit bureau in this case, especially the compliance department to better target any potential risk or issue and eventually control the risk.

The blog can be break into three parts:

1. count the word and get the words' frequency
2. calculate the ratio (between word frequency of individual agency and word frequency of the entire complaint lists) and get the words with the highest ratio value
3. improve the ratio by excluding common words related to company' name

There are 1,171,183 complaints(records) and 18 features (variables), but this blog, we will be only using the*** text attribute - Consumer complaint narrative*** After excluding the row with missing consumer complaint narrative, we are left 345,158 complaints(records).

Before we dive in our analysis, let's first check one complaint and get a sense

'I tried to sign up for a spending monitoring program and Capital One will not let me access my account through them'

Okay, looks reasonable.
Let's get started!

In [1]:
#Load library
import pandas as pd
import numpy as np
import re
from collections import Counter

In [2]:
# For dispaly purpose
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
   

In [3]:
df = pd.read_csv('Consumer_Complaints.csv')

In [4]:
# take a look the first five observation in the dataset
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217
1,01/19/2017,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,When my loan was switched over to Navient i wa...,NaN,"Navient Solutions, LLC.",LA,NaN,NaN,Consent provided,Web,01/19/2017,Closed with explanation,Yes,No,2296496
2,04/06/2018,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,I tried to sign up for a spending monitoring p...,NaN,CAPITAL ONE FINANCIAL CORPORATION,VA,NaN,Older American,Consent provided,Web,04/06/2018,Closed with explanation,Yes,NaN,2866101
3,06/08/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06/10/2014,Closed with explanation,Yes,Yes,885638
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760


In [5]:
print('Complain data set shape: ', df.shape)

Complain data set shape:  (1171183, 18)


There are 1,171,183 complaints(records) and 18 features (variables).

After we get a sense of the dataset, let's dive in the text attribute - consumer complaint narrative

first, take a look the complaint narrative example

In [6]:
df['Consumer complaint narrative'][2]

'I tried to sign up for a spending monitoring program and Capital One will not let me access my account through them'

# Step 1: count the word frequency

In [7]:
# Subset the data by company
EQU = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='EQUIFAX, INC.')]
EXP = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='Experian Information Solutions Inc.')]
TRU = df[(df['Consumer complaint narrative'].notnull())&(df['Company']=='TRANSUNION INTERMEDIATE HOLDINGS, INC.')]
TOTAL = df[(df['Consumer complaint narrative'].notnull())]

# Take a look how many complaint related to each company
print(len(EQU),'complaints related to Equifax')
print(len(EXP),'complaints related to Experian')
print(len(TRU),'complaints related to TransUnion')
print(len(TOTAL),'complaints in Total')

33470 complaints related to Equifax
27621 complaints related to Experian
27135 complaints related to TransUnion
345158 complaints in Total


In [8]:
# Create a empty Counter Object for the next step counting
EQU_counts = Counter()
EXP_counts = Counter()
TRU_counts = Counter()
TOTAL_counts = Counter()

EQU_lt = EQU['Consumer complaint narrative'].tolist()
EXP_lt = EXP['Consumer complaint narrative'].tolist()
TRU_lt = TRU['Consumer complaint narrative'].tolist()
TOTAL_lt = TOTAL['Consumer complaint narrative'].tolist()

In [9]:
#loop over all the words in the complaints and add up the counts
def count_word(complaints,word_counts):
    for i in range(len(complaints)):
        for word in re.split(r'\W+',  complaints[i]):
            word_counts[word] +=1

In [10]:
# count the word for each company's complaint lists
count_word(EQU_lt,EQU_counts)
count_word(EXP_lt,EXP_counts)
count_word(TRU_lt,TRU_counts)
count_word(TOTAL_lt,TOTAL_counts)

In [11]:
# extract the most common 10 words used in each company's complaint
EQU_counts_10 = EQU_counts.most_common(10)
EXP_counts_10 = EXP_counts.most_common(10)
TRU_counts_10 = TRU_counts.most_common(10)
TOTAL_counts_10 = TOTAL_counts.most_common(10)

In [12]:
# convert to dataframe for display
EQU_df = pd.DataFrame({'most 10 common (EQU)':EQU_counts_10})
EXP_df = pd.DataFrame({'most 10 common (EXP)':EXP_counts_10})
TRU_df = pd.DataFrame({'most  10 common (TRU)':TRU_counts_10})
Total_df = pd.DataFrame({'most 10 common (Total)':TOTAL_counts_10})

display('EQU_df', 'EXP_df', 'TRU_df', 'Total_df')

,most 10 common (EQU)
0,"(XXXX, 353350)"
1,"(the, 165262)"
2,"(to, 146177)"
3,"(I, 146117)"
4,"(and, 123997)"
5,"(my, 100196)"
6,"(a, 81678)"
7,"(of, 80311)"
8,"(XX, 73800)"
9,"(that, 71073)"


As the side by side comparison shown above for the top 10 common words used in different companies and entire complaint lists, common words like "the" is ranked the top among Equifax, Experian, TransUnion, and the Entire complaints lists.

Instead of finding the common words in the Equifax or Experian or TransUnion complaints, ***what we really want is those words that are shown far more often among one company's complain rather than the total complaint list. In other word, what complaint key word is concentrated uniquely for this company.***


# Step2: Calculate the frequent ratio

To accomplish this, we' ll need to calculate the word usage ratio between individual company and the entire list.

Use "the" as an example,

$$FreqRatio = \frac{Count_{EQU}['the']}{(Count_{TOTAL}['the'] + 1)}$$

Note: the "+ 1 " here is added in case the TOTAL_counts for some words is zero.

Dividing the company specific count on a word by the total count of the same word, we can let the company unique complaint key word stands out, and suppress the importance of common words like "the".

Let's calculated the ratio!

In [13]:
# Again, create Counter object for ratio calculation
EQU_ratios = Counter()
EXP_ratios = Counter()
TRU_ratios = Counter()

In [14]:
def calculate_ratio(word_counts,ratios):
    for word in list(word_counts):
        ratio = word_counts[word] / float(TOTAL_counts[word]+1)
        ratios[word] = ratio

In [15]:
# calculate the ratio for each company's complaint words
calculate_ratio(EQU_counts,EQU_ratios)
calculate_ratio(EXP_counts,EXP_ratios)
calculate_ratio(TRU_counts,TRU_ratios)

In [16]:
# words with the highest ratio 
EQU_df = pd.DataFrame({'most_common (EQU)':EQU_ratios.most_common(10)})
EXP_df = pd.DataFrame({'most_common (EXP)':EXP_ratios.most_common(10)})
TRU_df = pd.DataFrame({'most_common (TRU)':TRU_ratios.most_common(10)})

display('EQU_df', 'EXP_df', 'TRU_df')

,most_common (EQU)
0,"(equifaxsecurity2017, 0.9981203007518797)"
1,"(TrustedID, 0.9900497512437811)"
2,"(Equifaxs, 0.98989898989899)"
3,"(Equifax, 0.9895129913689013)"
4,"(EQUIFAX, 0.9863325740318907)"
5,"(EquiFax, 0.9791666666666666)"
6,"(Eqifax, 0.9777777777777777)"
7,"(Exquifax, 0.9777777777777777)"
8,"(Intruders, 0.9761904761904762)"
9,"(Equfax, 0.975)"


As the side by side comparison shown above this time, we started to see the differentiation. Unfortunately, biggest differentiation in the complaints is the company name, which is again not we really interested about, we want to find what's the real financial service issue in those complaints!

But we already very close to the answer we are trying to answer, just need one more step - leave out those word related to company names.

To accomplish this, we'll skip the counting when it is company related name.

# Step 3: improve the ratio 

One simply way is just say if any time we see 'Equifax', for example, we skip counting the word frequency, so that this word will be automatically, have a zero frequency, and showing in our most common list.

However, as we can see in the result above, when customer wrote complaint, they misspelled a lot. So by excluding just 'Equifax', is not gonna get us what we want.

Two ways of dealing with it:

1. manually summarize the misspelled pattern
2. using the library FuzzyWuzzy to implement a fuzzy matching

An example below is demonstrating how the fuzzywuzzy works. 
basically, it calculate a distance (called Levenshtein distance) to measure the difference between two sequence, in our case, two words. the higher the score, the more closer there two sequence are.


In [17]:
# illustrate how the fuzzywuzzy.process work
from fuzzywuzzy import process

misspelled1 = 'Exquifaax'
misspelled2 = 'Exclude'
match = ['equifax']
fuzzy_score1 = process.extract(misspelled1, match)
fuzzy_score2 = process.extract(misspelled2, match)
print(fuzzy_score1)
print(fuzzy_score2)

[('equifax', 88)]
[('equifax', 29)]


C:\Users\chenl\Anaconda3\envs\py36\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Back to our example, we for sure want to exclude those with high fuzzy score, which are those misspelled.

The pros and cons for the two approach I mentioned above:

* the manually summarizing is fast, but can't exclude the fuzzy words
* the fuzzy matching is accurately excluding those fuzzy word, but the code can take very long time to run, since it has to compute the fuzzy score for every word in the complaint list.

Therefore,  A mixed approach that combined first and second option can by more reasonable and leads to faster and more accurate result.The mixed approach is, if a word contain certain string like  "eq" in the Equifax complaints, those words are far more likely is to be a misspelled 'Equifax '. 

Therefore:

* if the condition like containing “eq” is met, fuzzy matching function will be called. 
* if condition is not matched, simply count the word with out any other processing.
* when the fuzzy matching function is called, is the fuzzy_score is greater than 85, skip the counting, since it is probably a misspelled 'Equifax', which we don't care that much


For the implementation, everything else keep the same as shown previously, and only revise this count_word function, since we need to update our new logic


In [18]:
# Create a empty Counter Object for the next step counting
EQU_counts2 = Counter()
EXP_counts2 = Counter()
TRU_counts2 = Counter()

# Again, create Counter object for ratio calculation
EQU_ratios2 = Counter()
EXP_ratios2 = Counter()
TRU_ratios2 = Counter()


In [19]:
#loop over all the words in the complaints and add up the counts
def count_word_new(word_lt,word_cnt,c_name_int,c_name):
    for i in range(len(word_lt)):
        lt = filter(None, re.split(r'\W+',  word_lt[i]))
        for word in lt:
            if word.lower().find(c_name_int) != -1:
                fuzzy_score = process.extract(word, c_name)[0][1]
                if fuzzy_score>=80:
                     continue
                else:
                    word_cnt[word] += 1

            else:
                word_cnt[word] += 1

In [20]:
count_word_new(EQU_lt,EQU_counts2,"eq",["Equifax"])
count_word_new(EXP_lt,EXP_counts2,"expe",["Experian"])
count_word_new(TRU_lt,TRU_counts2,"tran",["TransUnion"])

In [21]:
calculate_ratio(EQU_counts2,EQU_ratios2)
calculate_ratio(EXP_counts2,EXP_ratios2)
calculate_ratio(TRU_counts2,TRU_ratios2)

EQU_df = pd.DataFrame({'most_common (EQU)':EQU_ratios2.most_common(10)})
EXP_df = pd.DataFrame({'most_common (EXP)':EXP_ratios2.most_common(10)})
TRU_df = pd.DataFrame({'most_common (TRU)':TRU_ratios2.most_common(10)})

display('EQU_df', 'EXP_df', 'TRU_df')

,most_common (EQU)
0,"(TrustedID, 0.9900497512437811)"
1,"(Exquifax, 0.9777777777777777)"
2,"(Intruders, 0.9761904761904762)"
3,"(segmentation, 0.9591836734693877)"
4,"(realleges, 0.9565217391304348)"
5,"(trustedidpremier, 0.9473684210526315)"
6,"(cyberattack, 0.9438202247191011)"
7,"(Trusted, 0.9404761904761905)"
8,"(trustedid, 0.9375)"
9,"(2013Correct, 0.9375)"


Those misspelled decreased a lot, although not all!

Based on this complaint list, we can summarize

* Equifax's complaint are mostly concentrated in TrustedID, Intruders, segmentation, re alleges, cyber attack and 2013correct.
* Experian's complaints are mostly concentrated in Geographical, Credit Works, free credit report, Inquiry, Delinquency.
* Trans Union's complaints are mostly concentrated in 3rd party info, Libellant, LLCConsumer, Inquiry, Financing.


That's it. 😃 

We answered our question - what are the top 10 key complaint word that the top 3 credit bureau agencies (Equifax, Experian, and TransUnion) received ?

Hopefully, this blog demonstrate the power of the text mining, even it's a simple one, in helping compliance department in the financial industry to gain insights on what are the potential risk based on the customer text data.

Appendix
link for the dataset: https://www.consumerfinance.gov/data-research/consumer-complaints/<br>
Specail thanks to the display code from: https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html
